<a href="https://colab.research.google.com/github/prabh1010/Intel_img_classifier/blob/main/img_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
!gdown https://drive.google.com/uc?id=1cmf6sgyKpJx0i6svq6NXNNbCH7WsSAVv
#https://drive.google.com/file/d/1cmf6sgyKpJx0i6svq6NXNNbCH7WsSAVv/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1cmf6sgyKpJx0i6svq6NXNNbCH7WsSAVv
To: /content/archive.zip
363MB [00:08, 45.2MB/s]


In [ ]:
pwd

'/content'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/archive.zip

Archive:  /content/archive.zip
replace seg_pred/seg_pred/10004.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

cpu


In [ ]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [ ]:
train_path='/content/seg_train/seg_train'
test_path='/content/seg_test/seg_test'
train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),batch_size=8,shuffle=True)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),batch_size=4,shuffle=True)


In [ ]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
print(classes)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [ ]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [ ]:
model=ConvNet(num_classes=6).to(device)

In [ ]:
optimizer=Adam(model.parameters(),lr=0.01,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [ ]:
num_epochs=60

In [ ]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [ ]:
print(train_count,test_count)

14034 3000


In [ ]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'/content/model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(10.0097) Train Accuracy: 0.4624483397463303 Test Accuracy: 0.49633333333333335
Epoch: 1 Train Loss: tensor(1.1482) Train Accuracy: 0.55956961664529 Test Accuracy: 0.5863333333333334
Epoch: 2 Train Loss: tensor(1.0208) Train Accuracy: 0.6001140088356848 Test Accuracy: 0.618
Epoch: 3 Train Loss: tensor(0.9528) Train Accuracy: 0.6234858201510617 Test Accuracy: 0.6236666666666667
Epoch: 4 Train Loss: tensor(0.9370) Train Accuracy: 0.6360267920763859 Test Accuracy: 0.6223333333333333
Epoch: 5 Train Loss: tensor(0.8861) Train Accuracy: 0.6597548810032777 Test Accuracy: 0.585
Epoch: 6 Train Loss: tensor(0.8597) Train Accuracy: 0.6750748182984181 Test Accuracy: 0.6463333333333333
Epoch: 7 Train Loss: tensor(0.8394) Train Accuracy: 0.6770699729229015 Test Accuracy: 0.6746666666666666
Epoch: 8 Train Loss: tensor(0.8196) Train Accuracy: 0.6866894684338036 Test Accuracy: 0.6863333333333334
Epoch: 9 Train Loss: tensor(0.8089) Train Accuracy: 0.6934587430525866 Test Accur

In [ ]:
pwd

'/content'

In [ ]:
!gdown https://drive.google.com/uc?id=1Ws6Cdp8WmUMiJyNyCpTYunVIrhI7SMGR
#https://drive.google.com/file/d/1Ws6Cdp8WmUMiJyNyCpTYunVIrhI7SMGR/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1Ws6Cdp8WmUMiJyNyCpTYunVIrhI7SMGR
To: /content/model.zip
4.36MB [00:00, 38.5MB/s]


In [ ]:
!unzip /content/model.zip

Archive:  /content/model.zip
 extracting: archive/data.pkl        
 extracting: archive/data/94341636305840  
 extracting: archive/data/94341636305920  
 extracting: archive/data/94341636306000  
 extracting: archive/data/94341636306080  
 extracting: archive/data/94341636306160  
 extracting: archive/data/94341636306320  
 extracting: archive/data/94341636306400  
 extracting: archive/data/94341636306480  
 extracting: archive/data/94341636306560  
 extracting: archive/data/94341636306640  
 extracting: archive/data/94341636306720  
 extracting: archive/data/94341636306800  
 extracting: archive/data/94341636306880  
 extracting: archive/data/94341636307040  
 extracting: archive/data/94341636307120  
 extracting: archive/data/94341636307280  
 extracting: archive/data/94344072167904  
 extracting: archive/data/94344072389648  
 extracting: archive/version         


In [ ]:
model.eval()

test_accuracy=0.0
for i, (images,labels) in enumerate(test_loader):
    if torch.cuda.is_available():
        images=Variable(images.cuda())
        labels=Variable(labels.cuda())
        
    outputs=model(images)
    _,prediction=torch.max(outputs.data,1)

In [ ]:
torch.max(outputs.data,1)

torch.return_types.max(values=tensor([0.0719, 0.0330, 0.0410, 0.0636]), indices=tensor([5, 5, 5, 5]))

In [ ]:
print(labels.data)

tensor([1, 2, 0, 5])
